# The Important Notes

## Introduction
This might be not amazing as you expected. However, the jupyter notebooks could leads us into an infinite world of knowledge as has been mentioned in [the official documentation](https://github.com/jupyter/jupyter/wiki/A-gallery-of-interesting-Jupyter-Notebooks) of Jupyter Notebooks!

### Statistics and Probability
[Stat110.net](https://projects.iq.harvard.edu/stat110) is the official web of course Statistics 110 by Joe Blitzstein from Harvard University.
* #1 : Introduction to Statistics. Statistics is the science of uncertainty, while Math is the science of certainty. He also explains the Binomial Probability (with or w/o order), Mosteller and Wallace, [Fermat and Pascal on Probability](https://www.york.ac.uk/depts/maths/histstat/pascal.pdf) in reponse to the gambler's question about predicting the outcome which results the Pascal's Triangle, and recommends IQSS of Harvard University to get a further comprehensive understanding about statistics. 
    * Pascal's Wager : If he believes in God and God does exist, there will be an infinite rewards! as shown in [this video](http://factmyth.com/factoids/probability-theory-was-invented-to-solve-a-gambling-problem/) *Look at the video, it really inspires us!*
    

### Machine Learning
[Chakri Cherukuri](https://github.com/ChakriCherukuri/mlviz) is quatitative expert of Bloomberg that giving us a comprehensive understanding about how to quantify finance.

### Linear Regression

The code below is trying to perform a linear regression

In [1]:
import numpy as np

from ipywidgets import Button, VBox, HBox, Label
import bqplot.pyplot as plt
from bqplot import LinearScale

In [2]:
def linreg(x, y):
    '''
    computes intercept and slope for a simple
    ols regression
    '''
    b = np.cov(x, y)[0, 1] / np.var(x)
    a = np.mean(y) - b * np.mean(x)
    return a, b

In [3]:

x = np.linspace(-10, 10, 50)
y = 5 * x + 7 + np.random.randn(50) * 20

def update_regline(*args):
    # update the y attribute of the reg_line with 
    # the results of running the ols regression on 
    # x and y attributes of the scatter plot
    a, b = linreg(scatter.x, scatter.y)
    reg_line.y = a + b * reg_line.x
    
    # update the equation label
    equation_label.value = eqn_tmpl.format(a, b)

# Add a scatter plot and a regression line on the same figure
axes_options = {'x': {'label': 'X'},
                'y': {'label': 'Y'}}
fig = plt.figure(title='Linear Regression', animation_duration=1000)
                 
plt.scales(scales={'x': LinearScale(min=-30, max=30),
                   'y': LinearScale(min=-150, max=150)})

scatter = plt.scatter(x, y, colors=['orangered'], default_size=100, 
                      enable_move=True, interactions={'click': 'add'},
                      stroke='black')
reg_line = plt.plot(np.arange(-30, 31), [], 'g', stroke_width=8,
                    opacities=[.5], axes_options=axes_options)

fig.layout.width = '800px'
fig.layout.height = '550px'

reset_button = Button(description='Reset', button_style='success')
reset_button.layout.margin = '0px 30px 0px 60px'

eqn_tmpl = 'Regression Line: ${:.2f} + {:.2f}x$'
equation_label = Label()

def reset_points(*args):
    '''
    resets the scatter's x and y points 
    to the original values
    '''
    with scatter.hold_trait_notifications():
        scatter.x = x
        scatter.y = y

# on button click reset the scatter points
reset_button.on_click(lambda btn: reset_points())
# recompute reg line when new points are added
scatter.observe(update_regline, ['x', 'y'])

# compute the reg line
update_regline(None)

In [4]:
VBox([fig, HBox([reset_button, equation_label])])

### Gradient Descent

The code below is trying to perform a gradient descent.

In [5]:
from time import sleep

import numpy as np

from ipywidgets import *
import bqplot.pyplot as plt
from bqplot import Toolbar

In [6]:
f = lambda x: np.exp(-x) * np.sin(5 * x)
df = lambda x: -np.exp(-x) * np.sin(5 * x) + 5 * np.cos(5 *x) * np.exp(-x)

In [7]:
x = np.linspace(0.5, 2.5, 500)
y = f(x)

In [8]:
def update_sol_path(x, y):
    with sol_path.hold_sync():
        sol_path.x = x
        sol_path.y = y
    
    with sol_points.hold_sync():
        sol_points.x = x
        sol_points.y = y

In [9]:
def gradient_descent(x0, f, df, eta=.1, tol=1e-6, num_iters=10):
    x = [x0]
    i = 0
    
    while i < num_iters:
        x_prev = x[-1]
        grad = df(x_prev)
        x_curr = x_prev - eta * grad
        x.append(x_curr)
        sol_lbl.value = sol_lbl_tmpl.format(x_curr)
        sleep(.5)
        
        update_sol_path(x, [f(i) for i in x])
        
        if np.abs(x_curr - x_prev) < tol:
            break
        i += 1

In [10]:
txt_layout = Layout(width='150px')
x0_box = FloatText(description='x0', layout=txt_layout, value=2.4)
eta_box = FloatText(description='Learning Rate', 
                    style={'description_width':'initial'}, 
                    layout=txt_layout, value=.1)

go_btn = Button(description='GO', button_style='success', layout=Layout(width='50px'))
reset_btn = Button(description='Reset', button_style='success', layout=Layout(width='100px'))

sol_lbl_tmpl = 'x = {:.4f}'
sol_lbl = Label()
sol_lbl.layout.width = '300px'

# plot of curve and solution
fig_layout = Layout(width='720px', height='500px')
fig = plt.figure(layout=fig_layout, title='Gradient Descent', display_toolbar=True)
fig.pyplot = Toolbar(figure=fig)

curve = plt.plot(x, y, colors=['dodgerblue'], stroke_width=2)
sol_path = plt.plot([], [], colors=['#ccc'], opacities=[.7])
sol_points = plt.plot([], [], 'mo', default_size=20)

def optimize():
    f.marks = [curve]
    gradient_descent(x0_box.value, f, df, eta=eta_box.value)

def reset():
    curve.scales['x'].min = .4
    curve.scales['x'].max = 2.5
    
    curve.scales['y'].min = -.5
    curve.scales['y'].max = .4
    sol_path.x = sol_path.y = []
    sol_points.x = sol_points.y = []
    sol_lbl.value = ''
    
go_btn.on_click(lambda btn: optimize())
reset_btn.on_click(lambda btn: reset())

final_fig = VBox([fig, fig.pyplot], 
                 layout=Layout(overflow_x='hidden'))
HBox([final_fig, VBox([x0_box, eta_box, go_btn, reset_btn, sol_lbl])])